In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('drive/MyDrive/DM/SunsBucksClean.csv', sep=',', engine='python', error_bad_lines=False)

In [4]:
data.shape

(246239, 36)

In [5]:
data1=data.groupby(['user_mentions']).count().reset_index()

In [6]:
data.shape

(246239, 36)

In [7]:
# parte di estrazione attributi

In [8]:
data['user_mentions'][1].index('id_str')

80

In [9]:
data['user_mentions'][1][80:].split(',')[0][10:-1]

'18144089'

In [10]:
type(data['user_mentions'][1])

str

In [ ]:
nome=[]
for i in range(len(data.user_mentions)):
  if type(data['user_mentions'][i]) == str:
    if data['user_mentions'][i].find('id_str')!=-1:
      indice=data['user_mentions'][i].index('id_str')
      nome.append(data['user_mentions'][i][indice:].split(',')[0])
    else:
      nome.append('')
  else:
    nome.append('')

In [ ]:
nome_completo=[]
for i in range(len(data.user_mentions)):
  if type(data['user_mentions'][i]) == str:
    if data['user_mentions'][i].find('screen_name')!=-1:
      indice1=data['user_mentions'][i].index('screen_name')
      nome_completo.append(data['user_mentions'][i][indice1:].split(',')[0])
    else:
      nome_completo.append('')
  else:
    nome_completo.append('')

In [ ]:
nome_completo[0].split(':')[1]

" 'Suns'"

In [ ]:
nome1=[]
for i in range(len(nome)):
  if nome[i] != '':

   nome1.append(nome[i].split(':')[1])
  else:
    nome1.append('') 

In [ ]:
nome_completo1=[]
for i in range(len(nome_completo)):
  if nome_completo[i] != '':

   nome_completo1.append(nome_completo[i].split(':')[1])
  else:
    nome_completo1.append('') 

In [ ]:
data['user_mentions_screen_name']=nome_completo1
data['user_mentions_id']=nome1
# ora ho i due attributi che mi servono

In [ ]:
data=data[data['user_followers'] > 100] # numero minimo di follower per comparire nel grafo, per evitare bot

In [ ]:
data.head()

,Unnamed: 0,Unnamed: 0.1,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,retweeted,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,retweeted_status__user_id,retweeted_status_user_name,id_str,screen_name,sentiment,compound,emotion,positive,anger,anticipation,trust,surprise,negative,sadness,disgust,joy,fear,user_mentions_screen_name,user_mentions_id
0,0,0,60f74336b712538547cbf0fc,1417600743033360384,AZ,en,2021-07-20 23:42:09,suns 𝐍𝐁𝐀 𝐅𝐈𝐍𝐀𝐋𝐒 𝐆𝐀𝐌𝐄 𝟔 📺 abcnetwork youtubetv ...,0,1537,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'Suns', 'name': 'Phoenix Suns...","[{'text': 'RallyTheValley', 'indices': [120, 1...",NaN,NaN,1.417543e+18,1.848111e+07,Phoenix Suns,1146370740,boi_guapo,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.5106,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0,'Suns','18481113'
1,1,1,60f74338b712538547cbf0fd,1417600749182365700,"Stockholm, Sverige",en,2021-07-20 23:42:11,davidalangrier breaking news nba ref scott fos...,0,1036,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'davidalangrier', 'name': 'Da...","[{'text': 'NBAFinals', 'indices': [120, 130]}]",NaN,NaN,1.417601e+18,1.814409e+07,David Alan Grier,1061255281341054977,Ahmedberhan1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0,'davidalangrier','18144089'
2,2,2,60f7433ab712538547cbf0fe,1417600757872959492,By the Rich People,en,2021-07-20 23:42:13,wildcardsps k streams‼️ supoers beginning than...,0,2485,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'wildcardsprts', 'name': 'Wil...",[],NaN,NaN,1.417596e+18,1.142281e+18,WildCard Sports,239012690,Pastor_KevyKev,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0,'wildcardsprts','1142280818934583301'
3,3,3,60f7433bb712538547cbf0ff,1417600761500999681,"Charlotte, NC",en,2021-07-20 23:42:13,many postit notes,0,174837,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>",[],[],NaN,NaN,NaN,NaN,NaN,21362008,tomhaberstroh,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0,,
4,4,4,60f7433bb712538547cbf100,1417600763707154435,NaN,en,2021-07-20 23:42:14,johnypropz nba 🏀 july th ayton points like tai...,0,664,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'JohnnyPropz', 'name': 'Johnn...","[{'text': 'BettingExpert', 'indices': [82, 96]...",NaN,NaN,1.417601e+18,1.356723e+18,Johnny Props,1281401105398796288,JohnnyLockz,"{'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'comp...",0.3612,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0,'JohnnyPropz','1356722543995068416'


In [ ]:
df=data[['id_str','screen_name','user_mentions_id','user_mentions_screen_name','user_followers','text']]


In [ ]:
df=df.groupby(df.columns.tolist(),as_index=False).size().reset_index()
#df.columns=['id_str','retweeted_status__user_id','weight']
#df=df.drop(' index')
df=df.rename(columns={'id_str':'user.id_str','user_mentions_id':'user_id_ret','size':'weight'})
df

,index,user.id_str,screen_name,user_id_ret,user_mentions_screen_name,user_followers,text,weight
0,0,422,shellen,'2279776304','Giannis_An34',26123,nice work gianisan bucks feaheder,1
1,1,874,waynesutton,,,70233,🧐 thoughts,1
2,2,874,waynesutton,,,70235,bucks nbafinals,1
3,3,874,waynesutton,'15900167','Bucks',70232,bucks,1
4,4,874,waynesutton,'18481113','Suns',70232,realized going suns coach monty williams team ...,1
...,...,...,...,...,...,...,...,...
183409,183409,1417583932934135813,KhashIn6ix,'15900167','Bucks',147,bucks backs wall multiple times absolutely loo...,1
183410,183410,1417583932934135813,KhashIn6ix,'15900167','Bucks',147,bucks dogs dogs man motorolaus feaheder,1
183411,183411,1417583932934135813,KhashIn6ix,'172580748','OldTakesExposed',145,oldtakesexposed feaheder nbaplayofs,1
183412,183412,1417583932934135813,KhashIn6ix,'25319414','NBATV',144,nbatv gianis ’ resume stacked 🔥 nbafinals,1


In [ ]:
df1=df.groupby('user_mentions_screen_name')['weight'].sum().reset_index()

In [ ]:
df1=df1[df1['weight'] > 1000] # numero minimo di menzioni totali per avere la label nel grafo

In [ ]:
df1=df1.drop(0)

In [ ]:
len(df1)

14

In [ ]:
lista=df1.user_mentions_screen_name.to_list()

In [ ]:
labels=[]
for i in df.user_mentions_screen_name:
  if i in lista:
    labels.append(i)
  else:
    labels.append(' ')

In [ ]:
labels = pd.Series(labels)
df['labels']=labels

In [ ]:
df=df.drop(columns=['index'])

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df = df[df.user_mentions_screen_name != '']
df=df.rename(columns={'user_id_ret':'user_mentions_id'})
df['user_mentions_screen_name'] = df.user_mentions_screen_name.str.slice(start=2, stop=-1)
df['user_mentions_id'] = df.user_mentions_id.str.slice(start=2, stop=-1)
df['labels'] = df.labels.str.slice(start=2, stop=-1)
df=df.rename(columns={'user.id_str':'source','user_mentions_id':'target'})
df['source']=df['source'].round(0).astype(int)
df.head(50)

,source,screen_name,target,user_mentions_screen_name,user_followers,text,weight,labels
0,422,shellen,2279776304,Giannis_An34,26123,nice work gianisan bucks feaheder,1,Giannis_An34
3,874,waynesutton,15900167,Bucks,70232,bucks,1,Bucks
4,874,waynesutton,18481113,Suns,70232,realized going suns coach monty williams team ...,1,Suns
9,12225,satya,19085082,BallySportWI,5105,balyspowi gianis staying milwaukee could 've g...,1,
13,12997,ylynfatt,15900167,Bucks,449,congrats bucks fought back win straight champi...,1,Bucks
14,20503,pageman,15053533,mdesisti,4078,mdesisti wow pics tc i'l post congrats bucks ride,1,
15,20503,pageman,15900167,Bucks,4078,bucks people told ca n't unmake throws made fr...,1,Bucks
16,20503,pageman,22178780,NBAonTNT,4078,nbaont best best 💪 gianis antetokounmpo nbafin...,1,NBAonTNT
17,20503,pageman,366781966,LilySZhao,4078,lilyszhao bucks gianis antetokounmpo 🤣 “ n't p...,1,
18,20503,pageman,912359571947085824,RealBillRussell,4078,realbilrusel sorry take covid pass year gianis...,1,RealBillRussell


In [ ]:
#df['source']=df['source'].astype(str)
#df['target']=df['target'].astype(str)
df['source'] = df['source'].astype('|S80')
df['target'] = df['target'].astype('|S80')

In [ ]:
#df=df[(df['user_mentions_screen_name']=='Giannis_An34')|(df['user_mentions_screen_name']=='CP3')|(df['user_mentions_screen_name']=='Khris22m')|(df['user_mentions_screen_name']=='DevinBook')]
#df

,source,screen_name,target,user_mentions_screen_name,user_followers,text,weight,labels
0,b'422',shellen,b'2279776304',Giannis_An34,26123,nice work gianisan bucks feaheder,1,Giannis_An34
28,b'43693',kaimac,b'2279776304',Giannis_An34,5669,casual dominating star performance gianisan dr...,1,Giannis_An34
29,b'43693',kaimac,b'2279776304',Giannis_An34,5669,one time greatest finals performances right un...,1,Giannis_An34
34,b'654203',IOL,b'2279776304',Giannis_An34,525083,gianisan scored points added rebounds five blo...,1,Giannis_An34
73,b'963611',heatxsink,b'53853197',CP3,673,come cp … tonight night ralythevaley,1,CP3
...,...,...,...,...,...,...,...,...
182693,b'1404241860592377862',CelticsToxic,b'53853197',CP3,104,smoking pack tonight cp milvsphx scotfoster nb...,1,CP3
182699,b'1404330661914320897',Kidus_99,b'53853197',CP3,225,everyone talking shit cp real problem team ayt...,1,CP3
182963,b'1408144912864927749',NFTGOATS,b'2279776304',Giannis_An34,197,huge congrats gianisan winning first title wel...,1,Giannis_An34
183110,b'1410428148622135301',DRecEdits,b'2279776304',Giannis_An34,745,night gianisan historic point night bucks take...,1,Giannis_An34


In [ ]:
from google.colab import files
df.to_csv('Dataset_Gephi_mentions.csv', index=False)
files.download('Dataset_Gephi_mentions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.tail()

,source,screen_name,target,user_mentions_screen_name,weight,labels
43658,1413064261782360066,JeronymoBoss2,912359571947085824,RealBillRussell,1,
43659,1413171636241518594,JBlaze69er,926868226654404608,Nance726,1,
43660,1414429021086093313,yesisworld,168541923,ACTforAmerica,1,
43661,1415701981994360835,itslauryn21,1245300836596678661,Freshdipo1,1,
43662,1415701981994360835,itslauryn21,1247266037965389825,Hayomide_1,1,


In [ ]:
df.dtypes

source                       object
screen_name                  object
target                       object
user_mentions_screen_name    object
weight                        int64
labels                       object
dtype: object